In [1]:
import numpy as np

from finn.util.visualization import showInNetron

from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes

from qonnx.custom_op.registry import getCustomOp
from qonnx.core.datatype import DataType

import os
import pandas as pd
import my_metrics as metrics

# Load Original and OOC Model

In [2]:
ori_filename = 'BED_classifier__best_mean_F1__QONNX.onnx'
ori_bipolar_filename = 'BED_classifier__best_mean_F1__AIMET_Balanced__BIPOLAR_Out__QONNX.onnx'

# PYNQ-Z1 full build folder
model_folder = '.'
tidy_up_filename = model_folder + '/step_tidy_up.onnx'
# finn_filename = model_folder + '/custom_step_streamline.onnx'

### Original Model: cleaned

In [3]:
ori_cleaned_filename = "Best_ori_cleaned.onnx"
ori_bipolar_cleaned_filename = "Best_ori_bipolar_cleaned.onnx"

qonnx_cleanup(ori_filename, out_file= ori_cleaned_filename)
qonnx_cleanup(ori_bipolar_filename, out_file= ori_bipolar_cleaned_filename)

In [4]:
showInNetron(ori_cleaned_filename)

Serving 'Best_ori_cleaned.onnx' at http://0.0.0.0:8083


In [5]:
showInNetron(ori_bipolar_cleaned_filename)

Stopping http://0.0.0.0:8083
Serving 'Best_ori_bipolar_cleaned.onnx' at http://0.0.0.0:8083


### Show Tidy Up Model

In [6]:
showInNetron(tidy_up_filename)

Stopping http://0.0.0.0:8083
Serving './step_tidy_up.onnx' at http://0.0.0.0:8083


### Model Wrappers

In [7]:
ori_model = ModelWrapper(ori_cleaned_filename)
ori_bipolar_model = ModelWrapper(ori_bipolar_cleaned_filename)

tidy_up_model = ModelWrapper(tidy_up_filename)
# finn_model = ModelWrapper(finn_filename)

## Execute with Dummy IN

In [8]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 230, 230)) 
dummy_in = dummy_in.astype(np.float32)

# OOC needs (1, 224, 224, 3)
ooc_dummy_in = np.transpose(dummy_in, (0, 2, 3, 1))

# Print shapes
print(f'dummy_in shape: {dummy_in.shape}')
print(f'dummy_in transposed shape: {ooc_dummy_in.shape}')

dummy_in shape: (1, 3, 230, 230)
dummy_in transposed shape: (1, 230, 230, 3)


### Original Model

In [9]:
input_dict = {"global_in": (dummy_in / 255.) }
output_dict = oxe.execute_onnx(ori_model, input_dict)
produced_ori_qonnx = output_dict[list(output_dict.keys())[0]]
produced_ori_qonnx

array([[-12.896202,  -7.530959]], dtype=float32)

### Bipolar Original Model

In [10]:
output_dict = oxe.execute_onnx(ori_bipolar_model, input_dict)
produced_bipolar_ori_qonnx = output_dict[list(output_dict.keys())[0]]
produced_bipolar_ori_qonnx

array([[-1., -1.]], dtype=float32)

### Tidy Up

In [11]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(tidy_up_model, input_dict)
produced_tidy_up_qonnx = output_dict[list(output_dict.keys())[0]]
produced_tidy_up_qonnx

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_2_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_5_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_8_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  war

array([[-1., -1.]], dtype=float32)

In [12]:
# print(output_dict['global_out'][0])
# print(output_dict[list(output_dict.keys())[0]][0])

### FINN Model: streamline

In [13]:
# input_dict = {"global_in": dummy_in} #ooc_dummy_in}
# output_dict = oxe.execute_onnx(finn_model, input_dict)
# produced_tidy_up_qonnx = output_dict[list(output_dict.keys())[0]]
# produced_tidy_up_qonnx

# Eval with DFire MINI

In [14]:
DS_FOLDER = '../../datasets/dfire_mini/'
# DS_FOLDER += 'test/'
# DS_FOLDER += 'train/'
DS_FOLDER += '7ex/'

DS_IMGS = DS_FOLDER + 'images/'
DS_LABELS = DS_FOLDER + 'labels/'

In [15]:
IMGS_LIST = sorted([DS_IMGS + img_file for img_file in os.listdir(DS_IMGS)])

In [16]:
len(IMGS_LIST)

7

### QONNX model exported in Brevitas after training

In [17]:
# import importlib
# importlib.reload(metrics)

In [19]:
ori_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = ori_model,
    bipolar=False,
    divide_255=True) # Ori model does not have the preprocess included

Number of test samples: 7



Testing:  14%|████████████████████▋                                                                                                                            | 1/7 [00:00<00:01,  4.87it/s]

yhat before binarization: [ 5.69761   -3.0439763]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/PublicDataset00856.jpg: 
	label [1. 0.]
	pred [1. 0.]


Testing:  29%|█████████████████████████████████████████▍                                                                                                       | 2/7 [00:00<00:01,  4.09it/s]

yhat before binarization: [-4.831251 -7.766757]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB02490.jpg: 
	label [0. 0.]
	pred [0. 0.]


...

yhat before binarization: [ 4.2657394 -0.7782719]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09251.jpg: 
	label [1. 1.]
	pred [1. 0.]


Testing:  86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 6/7 [00:01<00:00,  4.08it/s]

yhat before binarization: [ 2.5809853 -4.3767433]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09620.jpg: 
	label [0. 0.]
	pred [1. 0.]
yhat before binarization: [-2.0153532 -0.7235942]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB10100.jpg: 
	label [0. 0.]
	pred [0. 0.]


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.09it/s]

yhat before binarization: [-0.2485823 -4.653549 ]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB11229.jpg: 
	label [1. 0.]
	pred [0. 0.]
Smoke -> TP: 3.0, FP: 1.0, TN: 2.0, FN: 1.0
Fire  -> TP: 1.0, FP: 0.0, TN: 5.0, FN: 1.0


In [20]:
ori_df = pd.DataFrame(ori_metrics)
ori_df

,Smoke,Fire,Mean
Accuracy,0.7143,0.8571,0.7857
Precision,0.7500,1.0000,0.8750
Recall,0.7500,0.5000,0.6250
F1,0.7500,0.6667,0.7084


### Bipolar Original Model

In [21]:
bipolar_ori_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = ori_bipolar_model,
    bipolar=True,
    divide_255=True) # Ori model does not have the preprocess included

Number of test samples: 7



Testing:  14%|████████████████████▋                                                                                                                            | 1/7 [00:00<00:00,  6.49it/s]

yhat before binarization: [ 1. -1.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/PublicDataset00856.jpg: 
	label [1. 0.]
	pred [1. 0.]


Testing:  29%|█████████████████████████████████████████▍                                                                                                       | 2/7 [00:00<00:00,  7.02it/s]

yhat before binarization: [-1. -1.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB02490.jpg: 
	label [0. 0.]
	pred [0. 0.]


...

yhat before binarization: [1. 1.]
yhat after binarization: [1. 1.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB04682.jpg: 
	label [1. 1.]
	pred [1. 1.]
yhat before binarization: [ 1. -1.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09251.jpg: 
	label [1. 1.]
	pred [1. 0.]


Testing:  71%|███████████████████████████████████████████████████████████████████████████████████████████████████████▌                                         | 5/7 [00:01<00:00,  4.56it/s]

yhat before binarization: [ 1. -1.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09620.jpg: 
	label [0. 0.]
	pred [1. 0.]


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.69it/s]

yhat before binarization: [-1. -1.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB10100.jpg: 
	label [0. 0.]
	pred [0. 0.]
yhat before binarization: [-1. -1.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB11229.jpg: 
	label [1. 0.]
	pred [0. 0.]
Smoke -> TP: 3.0, FP: 1.0, TN: 2.0, FN: 1.0
Fire  -> TP: 1.0, FP: 0.0, TN: 5.0, FN: 1.0


In [22]:
bipolar_df = pd.DataFrame(bipolar_ori_metrics)
bipolar_df

,Smoke,Fire,Mean
Accuracy,0.7143,0.8571,0.7857
Precision,0.7500,1.0000,0.8750
Recall,0.7500,0.5000,0.6250
F1,0.7500,0.6667,0.7084


### Tidy Up Model

In [23]:
clean_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = tidy_up_model,
    bipolar=True,
    divide_255=False) # Tidy Up model has the preprocess included

Number of test samples: 7



Testing:   0%|                                                                                                                                                         | 0/7 [00:00<?, ?it/s]/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_2_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_5_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/bas

yhat before binarization: [ 1. -1.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/PublicDataset00856.jpg: 
	label [1. 0.]
	pred [1. 0.]


Testing:  29%|█████████████████████████████████████████▍                                                                                                       | 2/7 [00:14<00:37,  7.47s/it]

yhat before binarization: [-1. -1.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB02490.jpg: 
	label [0. 0.]
	pred [0. 0.]


...

yhat before binarization: [ 1. -1.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09620.jpg: 
	label [0. 0.]
	pred [1. 0.]


Testing:  86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 6/7 [00:45<00:07,  7.60s/it]

yhat before binarization: [-1.  1.]
yhat after binarization: [0. 1.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB10100.jpg: 
	label [0. 0.]
	pred [0. 1.]


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.59s/it]

yhat before binarization: [-1. -1.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB11229.jpg: 
	label [1. 0.]
	pred [0. 0.]
Smoke -> TP: 3.0, FP: 1.0, TN: 2.0, FN: 1.0
Fire  -> TP: 1.0, FP: 1.0, TN: 4.0, FN: 1.0


In [24]:
# print(clean_metrics)

In [25]:
clean_df = pd.DataFrame(clean_metrics)
clean_df

,Smoke,Fire,Mean
Accuracy,0.7143,0.7143,0.7143
Precision,0.7500,0.5000,0.6250
Recall,0.7500,0.5000,0.6250
F1,0.7500,0.5000,0.6250


In [26]:
ori_df

,Smoke,Fire,Mean
Accuracy,0.7143,0.8571,0.7857
Precision,0.7500,1.0000,0.8750
Recall,0.7500,0.5000,0.6250
F1,0.7500,0.6667,0.7084


### Modify Tidy Up from Bipolar to Binary

In [27]:
Multithreshold_node = tidy_up_model.get_nodes_by_op_type("MultiThreshold")    
# Find MultiThreshold that FINN could not annotate datatype properly and set them to INT32
# It happens with all convolutions splitted by Spatial SVD
    # First conv is the one impacted: k=(3,1)
    # Second conv, k=(1,3), followed by BN and ReLU, is annotated properly
# for node in Multithreshold_node:
#     if tidy_up_model.get_tensor_datatype(node.input[1]) == "FLOAT32":
#         print(f'{node.name}: node with Float32 annotation')
#         tidy_up_model.set_tensor_datatype(node.input[1], DataType["INT32"])
#         print(f'{node.name}: changed to datatype {tidy_up_model.get_tensor_datatype(node.input[1])}')
# Change BIPOLAR end node to BINARY
for node in Multithreshold_node:
    node_inst = getCustomOp(node)
    if node_inst.get_nodeattr("out_dtype") == "BIPOLAR":
        node_inst.set_nodeattr("out_dtype", "BINARY")
        node_inst.set_nodeattr("out_scale", 1.0)
        node_inst.set_nodeattr("out_bias", 0.0)
        print(f'Node changed from BIPOLAR to BINARY, to fulfill standalone MultiThreshold requirement\n{node}')
        print("Set Output to BINARY Datatype")
        global_out_name = tidy_up_model.graph.output[0].name
        tidy_up_model.set_tensor_datatype(global_out_name, DataType["BINARY"])

Node changed from BIPOLAR to BINARY, to fulfill standalone MultiThreshold requirement
input: "Add_6_out0"
input: "MultiThreshold_19_param0"
output: "global_out"
name: "MultiThreshold_19"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "BINARY"
  type: STRING
}
attribute {
  name: "out_scale"
  f: 1.0
  type: FLOAT
}
attribute {
  name: "out_bias"
  f: 0.0
  type: FLOAT
}
domain: "qonnx.custom_op.general"

Set Output to BINARY Datatype


In [28]:
surgery_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = tidy_up_model,
    bipolar=True,
    divide_255=False) # Tidy Up model has the preprocess included

Number of test samples: 7



Testing:   0%|                                                                                                                                                         | 0/7 [00:00<?, ?it/s]/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_2_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_5_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/bas

yhat before binarization: [1. 0.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/PublicDataset00856.jpg: 
	label [1. 0.]
	pred [1. 0.]


Testing:  29%|█████████████████████████████████████████▍                                                                                                       | 2/7 [00:15<00:37,  7.53s/it]

yhat before binarization: [0. 0.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB02490.jpg: 
	label [0. 0.]
	pred [0. 0.]


...

yhat before binarization: [1. 0.]
yhat after binarization: [1. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB09620.jpg: 
	label [0. 0.]
	pred [1. 0.]


Testing:  86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                    | 6/7 [00:45<00:07,  7.66s/it]

yhat before binarization: [0. 1.]
yhat after binarization: [0. 1.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB10100.jpg: 
	label [0. 0.]
	pred [0. 1.]


Testing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.60s/it]

yhat before binarization: [0. 0.]
yhat after binarization: [0. 0.]
Image file: ../../datasets/dfire_mini/7ex/images/WEB11229.jpg: 
	label [1. 0.]
	pred [0. 0.]
Smoke -> TP: 3.0, FP: 1.0, TN: 2.0, FN: 1.0
Fire  -> TP: 1.0, FP: 1.0, TN: 4.0, FN: 1.0


In [29]:
surgery_df = pd.DataFrame(surgery_metrics)

In [30]:
print(f'Ori Metrics \n{ori_df}\n')
print(f'Ori Bipolar Metrics \n{bipolar_df}\n')
print(f'Tidy up Metrics \n{clean_df}\n')
print(f'Surgery Metrics \n{surgery_df}\n')

Ori Metrics 
            Smoke    Fire    Mean
Accuracy   0.7143  0.8571  0.7857
Precision  0.7500  1.0000  0.8750
Recall     0.7500  0.5000  0.6250
F1         0.7500  0.6667  0.7084

Ori Bipolar Metrics 
            Smoke    Fire    Mean
Accuracy   0.7143  0.8571  0.7857
Precision  0.7500  1.0000  0.8750
Recall     0.7500  0.5000  0.6250
F1         0.7500  0.6667  0.7084

Tidy up Metrics 
            Smoke    Fire    Mean
Accuracy   0.7143  0.7143  0.7143
Precision  0.7500  0.5000  0.6250
Recall     0.7500  0.5000  0.6250
F1         0.7500  0.5000  0.6250

Surgery Metrics 
            Smoke    Fire    Mean
Accuracy   0.7143  0.7143  0.7143
Precision  0.7500  0.5000  0.6250
Recall     0.7500  0.5000  0.6250
F1         0.7500  0.5000  0.6250



In [31]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(tidy_up_model, input_dict)
produced_tidy_up_qonnx = output_dict[list(output_dict.keys())[0]]
produced_tidy_up_qonnx

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_2_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_5_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_8_out0 can't be represented with the set datatype annotation (INT4), they will be rounded to match the datatype annotation.
  war

array([[0., 0.]], dtype=float32)